# Step 1: Postprocessing of raw eddy-covariance data

In [1]:
#loading Reddy package
install.packages("../src/Reddy_0.0.0.9000.tar.gz",repos=NULL,source=TRUE)
library(Reddy)

Installing package into ‘/home/lauracma/R/x86_64-pc-linux-gnu-library/4.0’
(as ‘lib’ is unspecified)



In [12]:
#ec data files
dir_in="../data/ec-data_10Hz_raw"
files=list.files(dir_in,full.names=TRUE)
nf=length(files)

In [13]:
#allocate output
var_out=c("time","u_mean","v_mean","w_mean","ws_mean","wd_mean","T_mean","q_mean","co2_mean",
          "u_sd","v_sd","w_sd","T_sd","q_sd","co2_sd",
          "cov_uv","cov_uw","cov_vw","cov_wT","cov_qw","cov_co2w",
          "rot_angle1","rot_angle2","flag_stationarity","flag_w","flag_distortion")
nv=length(var_out)
dat=data.frame(array(NA,dim=c(nf,nv)))
colnames(dat)=var_out

In [17]:
#postprocessing per file (30 mins)
for (i in 1:nf) {
    tmp=read.table(files[i],sep=",",header=T)
    dat$time=tmp$X[1]
    #despiking
    tmp$T_degC=despiking(tmp$T_degC,-40,30)
    tmp$u_m.s=despiking(tmp$u_m.s,-25,25)
    tmp$v_m.s=despiking(tmp$v_m.s,-25,25)
    tmp$w_m.s=despiking(tmp$w_m.s,-5,5)
    #wind before rotation
    dat$ws_mean[i]=sqrt(mean(tmp$u_m.s,na.rm=T)^2+mean(tmp$v_m.s,na.rm=T)^2)
    dat$wd_mean[i]=atan2(mean(tmp$v_m.s,na.rm=T),mean(tmp$u_m.s,na.rm=T))
    #rotation
    rot_wind=rotate_double(tmp$u_m.s,tmp$v_m.s,tmp$w_m.s)
    tmp$u_m.s=rot_wind$u
    tmp$v_m.s=rot_wind$v
    tmp$w_m.s=rot_wind$w
    dat$rot_angle1[i]=rot_wind$theta
    dat$rot_angle2[i]=rot_wind$phi
    #averaging
    dat$u_mean[i]=mean(tmp$u_m.s,na.rm=T)
    dat$v_mean[i]=mean(tmp$v_m.s,na.rm=T)
    dat$w_mean[i]=mean(tmp$w_m.s,na.rm=T)
    dat$T_mean[i]=mean(tmp$T_degC,na.rm=T)
    dat$q_mean[i]=mean(tmp$H2O_ppt,na.rm=T)
    dat$co2_mean[i]=mean(tmp$CO2_ppm,na.rm=T)
    dat$u_sd[i]=sd(tmp$u_m.s,na.rm=T)
    dat$v_sd[i]=sd(tmp$v_m.s,na.rm=T)
    dat$w_sd[i]=sd(tmp$w_m.s,na.rm=T)
    dat$T_sd[i]=sd(tmp$T_degC,na.rm=T)
    dat$q_sd[i]=sd(tmp$H2O_ppt,na.rm=T)
    dat$co2_sd[i]=sd(tmp$CO2_ppm,na.rm=T)
    dat$cov_uw[i]=cov(tmp$u_m.s,tmp$w_m.s,use="pairwise.complete.obs")
    dat$cov_uv[i]=cov(tmp$u_m.s,tmp$v_m.s,use="pairwise.complete.obs")
    dat$cov_vw[i]=cov(tmp$v_m.s,tmp$w_m.s,use="pairwise.complete.obs")
    dat$cov_wT[i]=cov(tmp$T_degC,tmp$w_m.s,use="pairwise.complete.obs")
    dat$cov_qw[i]=cov(tmp$H2O_ppt,tmp$w_m.s,use="pairwise.complete.obs")
    dat$cov_co2w[i]=cov(tmp$CO2_ppm,tmp$w_m.s,use="pairwise.complete.obs")
    #SND correction
    #cov_wT_snd=SNDcorrection(tmp$u_m.s,tmp$v_m.s,tmp$w_m.s,tmp$T_degC)
    #flagging
    dat$flag_stationarity[i]=flag_stationarity(tmp$w_m.s,tmp$T_degC)
    dat$flag_w[i]=flag_w(dat$w_mean[i])
    dat$flag_distortion[i]=flag_distortion(tmp$u_m.s,tmp$v_m.s,dir_blocked=c(340,360))
}

In [18]:
head(dat)

,time,u_mean,v_mean,w_mean,ws_mean,wd_mean,T_mean,q_mean,co2_mean,u_sd,⋯,cov_uw,cov_vw,cov_wT,cov_qw,cov_co2w,rot_angle1,rot_angle2,flag_stationarity,flag_w,flag_distortion
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,2018-07-22 23:30:00,2.872084,-3.939352e-16,6.931732e-06,2.873170,-2.289284,15.86938,11.119238,377.2578,1.062143,⋯,-0.03118278,0.024897073,0.1052858,0.05995418,-0.09842712,-131.1663,0.3484284,1,0,NA
2,2018-07-22 23:30:00,2.864793,7.504684e-17,9.052527e-08,2.864538,-2.326045,16.55190,11.652726,376.1681,1.105122,⋯,-0.07402658,0.022741472,0.1600703,0.09478121,-0.14439163,-133.2726,0.7745417,1,0,NA
3,2018-07-22 23:30:00,3.996526,9.365292e-16,3.049021e-05,4.002522,-2.035397,17.05704,11.500354,375.3589,1.409495,⋯,-0.08568719,-0.001509985,0.1533514,0.07214777,-0.11741832,-116.6197,0.2888957,1,0,NA
4,2018-07-22 23:30:00,4.998016,-2.812846e-16,-2.391239e-06,4.997530,-1.977737,17.60447,10.427265,376.0030,1.289326,⋯,-0.11664376,-0.013668165,0.1556649,0.07221147,-0.09500221,-113.3160,0.3483120,1,0,NA
5,2018-07-22 23:30:00,4.879095,4.062636e-16,2.187117e-05,4.880696,-2.014489,18.08994,8.357570,376.3174,1.469949,⋯,-0.11678821,-0.007791092,0.1914488,0.11200968,-0.12587227,-115.4217,0.6531922,1,0,NA
6,2018-07-22 23:30:00,5.225037,-8.977916e-17,-6.972624e-06,5.223984,-2.261205,18.24207,6.835076,376.8836,1.319227,⋯,-0.12126287,0.018796952,0.1490271,0.08843027,-0.08421749,-129.5575,0.4471074,1,0,NA


In [19]:
saveRDS(dat,file="../data/ec-data_30min_processed/processed_data_example.rds")